1. Load drug data and process it

In [1]:
# Imports

import pandas as pd

In [123]:
# Load state drug data for 2008 - 2018

columns = ['State', 'Year', 'Product Name', 'Quarter Begin Date']
filenames = ['State_Drug_Utilization_Data_' + str(i) + '.csv' for i in range(2008, 2019)]

df = pd.concat(pd.read_csv(filename, usecols=columns) 
               for filename in filenames)

In [131]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39752112 entries, 0 to 2365964
Data columns (total 4 columns):
State                 object
Year                  int64
Product Name          object
Quarter Begin Date    object
dtypes: int64(1), object(3)
memory usage: 7.9 GB


In [134]:
# Convert non-numeric columns to category to save memory

df[['State', 'Product Name', 
    'Quarter Begin Date']] = df[['State', 
                                 'Product Name', 
                                 'Quarter Begin Date']].astype('category')

In [136]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39752112 entries, 0 to 2365964
Data columns (total 4 columns):
State                 category
Year                  int64
Product Name          category
Quarter Begin Date    category
dtypes: category(3), int64(1)
memory usage: 837.4 MB


In [139]:
df_samp = df.sample(10)
df_samp

,State,Year,Product Name,Quarter Begin Date
2607158,NY,2014,OLANZAPINE,01/01/2014
2485375,NM,2013,HYPERCARE,10/01/2013
1788539,MO,2013,DERMA-SMOO,04/01/2013
1145790,KY,2013,PREDNISONE,01/01/2013
3920314,RI,2014,Q-TUSSIN D,04/01/2014
1967635,XX,2014,PRILOSEC 4,10/01/2014
1347090,NY,2016,LEVOBUNOLO,04/01/2016
1064667,VA,2009,CENTURY SE,10/01/2009
1711818,MN,2014,SODIUM CIT,10/01/2014
1937982,MT,2013,RISPERIDON,10/01/2013


In [141]:
# TODO:
# 1. Convert quarter into 1-4
# 2. Do some frequency count plots
# 3. See which states use which drugs
# 4. Come up with some more questions asdas